훈련 인스턴스 가지고 비교하는 작업.  
학습 ㄴㄴ  
검증 단계에에서 관측값과 가장 근접한 훈련 관측 값을 비교  
유클리드 공간의 점과 점사이 직선거리를 사용  
다양한 평가지표 활용 가능  
거리에만 의존하여 차원의 저주에 의해 특징 개수가 늘어나면 성능 저하..

##### 특징을 표준화 하는 것이 중요

In [11]:
import pandas as pd
import numpy as np

In [4]:
breast_cancer = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data", header = None)

In [5]:
breast_cancer.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [6]:
breast_cancer.columns = ["id_number", "clump_thickness", "unif_cell_size", "unif_cell_shape", 
                         "marg_adhesion", "single_epith_cell_size","bare_nuclei", "bland_chromatin",
                         "normal_nucleoli", "mitoses", "class"]


In [7]:
breast_cancer.head()

,id_number,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_epith_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [8]:
breast_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id_number               699 non-null    int64 
 1   clump_thickness         699 non-null    int64 
 2   unif_cell_size          699 non-null    int64 
 3   unif_cell_shape         699 non-null    int64 
 4   marg_adhesion           699 non-null    int64 
 5   single_epith_cell_size  699 non-null    int64 
 6   bare_nuclei             699 non-null    object
 7   bland_chromatin         699 non-null    int64 
 8   normal_nucleoli         699 non-null    int64 
 9   mitoses                 699 non-null    int64 
 10  class                   699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [9]:
breast_cancer.isnull().values.sum()

0

In [12]:
breast_cancer.bare_nuclei = breast_cancer.bare_nuclei.replace("?", np.NaN)

In [13]:
breast_cancer.bare_nuclei = breast_cancer.bare_nuclei.fillna(breast_cancer.bare_nuclei.value_counts().index[0])

In [14]:
breast_cancer["cancer_ind"] = 0

In [15]:
breast_cancer.loc[breast_cancer["class"] == 4, "cancer_ind"] = 1

In [16]:
X = breast_cancer.drop(["id_number", "class", "cancer_ind"], axis=1)
y = breast_cancer.cancer_ind

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [19]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve 

In [23]:
y_pred = knn.predict(X_test_scaled)

In [24]:
accuracy_score(y_test, y_pred)

0.9761904761904762

In [25]:
confusion_matrix(y_test, y_pred)

array([[141,   2],
       [  3,  64]], dtype=int64)

In [26]:
roc_auc_score(y_test, y_pred)

0.9706189333055005

In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
grid_search = GridSearchCV(knn, {"n_neighbors": [1, 2, 3, 4, 5]}, n_jobs=-1, cv=7, scoring="roc_auc")
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=7, estimator=KNeighborsClassifier(n_neighbors=3), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5]}, scoring='roc_auc')

In [29]:
grid_search.best_params_

{'n_neighbors': 5}

In [30]:
knn_best = grid_search.best_estimator_

In [31]:
y_pred = knn_best.predict(X_test_scaled)


In [32]:
accuracy_score(y_test, y_pred)

0.9666666666666667

In [33]:
confusion_matrix(y_test, y_pred)

array([[141,   2],
       [  5,  62]], dtype=int64)

In [34]:
roc_auc_score(y_test, y_pred)


0.9556935601711722